In [ ]:
from datasets import load_dataset

ds = load_dataset('billingsmoore/Aggregated-bo-en', split='train')

## Train Tokenizer for Tibetan

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

Note that the tokenizer already contains some Tibetan tokens but not enough.

In [ ]:
enc = tokenizer.encode('ཡུན་རིང་དུས་ནས་ཆོས་ཀྱིས་བསྐྱངས་བའི་བུ། ')
tokenizer.decode(enc)

In [ ]:
from tokenizers import BertWordPieceTokenizer

new_tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False)

In [ ]:
new_tokenizer.train_from_iterator(
    ds['bo'],
    vocab_size=len(tokenizer.get_vocab()),
    min_frequency=3,
    show_progress=True,
    limit_alphabet=500
)

### Convert New Tokenizer to AutoTokenizer Format

In [ ]:
new_tokenizer.save_model('new_tokenizer')

In [ ]:
from transformers import BertTokenizerFast

# Load the trained tokenizer
fast_tokenizer = BertTokenizerFast(
    vocab_file="new_tokenizer/vocab.txt",
    do_lower_case=False
)

# Save in Hugging Face format
fast_tokenizer.save_pretrained("fast_tokenizer")

In [ ]:
from transformers import AutoTokenizer

# Load your new tokenizer
new_fast_tokenizer = AutoTokenizer.from_pretrained("fast_tokenizer")

In [ ]:
enc = new_fast_tokenizer.encode('ཡུན་རིང་དུས་ནས་ཆོས་ཀྱིས་བསྐྱངས་བའི་བུ། ')
new_fast_tokenizer.decode(enc)

## Train Embedding Model

### Load the Models

In [1]:
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    models
)
from transformers import AutoTokenizer

# Load teacher model
teacher_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cuda')

# load student model
word_embedding_model = models.Transformer('sentence-transformers/all-MiniLM-L6-v2')
word_embedding_model.tokenizer = AutoTokenizer.from_pretrained("fast_tokenizer")
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

student_model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device='cuda')

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


RuntimeError: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from datasets import load_dataset

ds = load_dataset('billingsmoore/Aggregated-bo-en', split='train')

# create teacher embeddings
def create_teacher_embeddings(batch):
    batch['teacher embeddings'] = teacher_model.encode(batch['en'])
    return batch

ds = ds.map(create_teacher_embeddings, batched=True)

ds[0]

### Define Loss Function

In [2]:
from sentence_transformers.losses import MSELoss

# 4. Define a loss function
loss = MSELoss(student_model)

NameError: name 'student_model' is not defined

### Specify Training Arguments

In [ ]:
# 5. Specify training arguments
args = SentenceTransformerTrainingArguments(
    output_dir="do-it-right?",
    num_train_epochs=1,
    auto_find_batch_size=True,
    learning_rate=2e-5,
    warmup_ratio=0.1,
    fp16=False,  # Set to False if you get an error that your GPU can't run on FP16
    eval_strategy="steps",
    eval_steps=100
)

### Define Evaluator

In [ ]:
from sentence_transformers.evaluation import MSEEvaluator
# Create an evaluator & evaluate the base model
dev_evaluator = MSEEvaluator(
    source_sentences=eval_dataset['english'],
    target_sentences=eval_dataset['tibetan'],
    teacher_model=teacher_model,
    name="stsb-dev",
    show_progress_bar=True
)
dev_evaluator(student_model)

{'stsb-dev_negative_mse': np.float32(-7.1650085)}

### Train the Model

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict({'english': ds['train']['en'], 'tibetan': ds['train']['bo'], 'label': ds['train']['teacher embeddings']})
eval_dataset = Dataset.from_dict({'english': ds['test']['en'], 'tibetan': ds['test']['bo'], 'label': ds['test']['teacher embeddings']})

In [ ]:
# 7. Create a trainer & train
trainer = SentenceTransformerTrainer(
    model=student_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    evaluator=dev_evaluator,
)
trainer.train()

/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


  0%|          | 0/98776 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'repeat'

In [ ]:
model.save('./fine-tuned-minilm')